# Least-square conformal map directly applied to nurbs surfaces

A use case of least square conformal maps is the mapping of mesh (triangle-surface) to a flat plane. This is normally used for textureing meshes. Given the fact that the least-square conformal map can produce the exact flattened surface for developable surface we try to perform the lscm method on spline-surfaces to optain an unwrapped representation of the nurbs-surface.

First a collocation method should be use to solve for conformity on nurbs surfaces. If this works for developable surfaces the extension to a weighted least-square method will be explored.

![mapping](mapping.svg)

## Conformal Energy

Let a suface be represented by a Nurbs Interpolation.

Then we have already a mapping from the U-system to the X system. We now want to find another map from U to L which minimize a certain criteria.
We introduce another ortho-normal coordinate-system $\eta$ which is perpendicular to a point on the surface. Now we write the Partial Differential Equation of the conformity for this specific point.

$$\begin{equation}
\frac{\partial l}{\partial \eta } - \frac{\partial m}{\partial \zeta } = 0\\
\frac{\partial l}{\partial \zeta } + \frac{\partial m}{\partial \eta } = 0
\end{equation}$$

## Implementation
### Some Basic NURBS derivatives:

The Surface is given by:

$$\begin{equation}
\mathbf{x}=\sum_{i=1}^{n}N_{i}(u,v)\cdot\mathbf{x_{i}}\\
\end{equation}$$

then the derivatives can be computed with:

$$\begin{equation}
\mathbf{x}_{u}=\frac{\partial\mathbf{x}}{\partial u}=\sum_{i=1}^{n}\frac{\partial N_{i}(u,v)}{\partial u}\cdot\mathbf{x_{i}}=\mathbf{b}_{u}\cdot\hat{\mathbf{X}}\\
\mathbf{x}_{v}=\frac{\partial\mathbf{x}}{\partial v}=\sum_{i=1}^{n}\frac{\partial N_{i}(u,v)}{\partial v}\cdot\mathbf{x_{i}}=\mathbf{b}_{v}\cdot\hat{\mathbf{X}}\\
\mathbf{b}_{u}=[\frac{\partial N_{1}}{\partial u},\frac{\partial N_{2}}{\partial u},...,\frac{\partial N_{n}}{\partial u}]\\
\mathbf{b}_{v}=[\frac{\partial N_{1}}{\partial v},\frac{\partial N_{2}}{\partial v},...,\frac{\partial N_{n}}{\partial v}]\\
\mathbf{b}_{ul}=[\mathbf{b}_{u},0,0,...,0]\quad \mathbf{b}_{vl}=[\mathbf{b}_{v},0,0,...,0]\\
\mathbf{b}_{um}=[0,0,...,0,\mathbf{b}_{u}]\quad \mathbf{b}_{vm}=[0,0,...,0,\mathbf{b}_{v}]
\end{equation}$$

### Find a local Coordinate System:

In this step we try to find a local normalized orthogonal system which has a $\boldsymbol{\xi}$ component normal to the surface.

$$\begin{equation}
\boldsymbol{\eta}=\frac{\mathbf{x}_u}{|\mathbf{x}_u|}\quad
\boldsymbol{\xi}=\frac{\boldsymbol{\eta}\times\mathbf{x}_v}{|\boldsymbol{\eta}\times\mathbf{x}_v|}\quad\boldsymbol{\zeta}=\boldsymbol{\xi}\times\boldsymbol{\eta}\\
\end{equation}$$

### Jacobi-matrix from U to $\eta$

$$\begin{equation}
\mathbf{J}=\left(\begin{array}{cc}
\frac{\partial\eta}{\partial u} & \frac{\partial\zeta}{\partial u}\\
\frac{\partial\eta}{\partial v} & \frac{\partial\zeta}{\partial v}
\end{array}\right)\quad\mathbf{J}^{-1}=\left(\begin{array}{cc}
\frac{\partial u}{\partial\eta} & \frac{\partial v}{\partial\eta}\\
\frac{\partial u}{\partial\zeta} & \frac{\partial v}{\partial\zeta}
\end{array}\right)\\
\frac{\partial\eta}{\partial u}=\mathbf{x}_{u}^t\cdot\boldsymbol{\eta}\quad
\frac{\partial\zeta}{\partial u}=\mathbf{x}_{u}^t\cdot\boldsymbol{\zeta}\\
\frac{\partial\eta}{\partial v}=\mathbf{x}_{v}^t\cdot\boldsymbol{\eta}\quad
\frac{\partial\zeta}{\partial v}=\mathbf{x}_{v}^t\cdot\boldsymbol{\zeta}\\
\mathbf{J}=\left(\begin{array}{c}\mathbf{x}_{u}^t\\\mathbf{x}_{v}^t\end{array}\right)
\left(\begin{array}{cc}\boldsymbol{\eta} & \boldsymbol{\zeta}\end{array}\right)
\end{equation}$$


### Equations

To find a conformal map of the surface the equation () have to be satisfied on all points of the surface. As the interpolation of the NURBS-surface have a endless number of unknowens, we do not have to form the equations on every point. We choose the number of collocation-points higher then the number of unknowens. This way we have to solve the equation in the least-square sense. If the surface is developable, all equations should be solved exactly, otherwise we solve for a minimum error. For every collocation point we can form these two equations:

$$\begin{equation}
\left(\mathbf{b}_{ul} \cdot J_{11}^{-1}+\mathbf{b}_{vl} \cdot J_{12}^{-1}-\mathbf{b}_{um} \cdot J_{21}^{-1}-\mathbf{b}_{vm} \cdot J_{22}^{-1}\right)\hat{\mathbf{L}}=0\\
\left(\mathbf{b}_{ul} \cdot J_{21}^{-1}+\mathbf{b}_{vl} \cdot J_{22}^{-1}+\mathbf{b}_{um} \cdot J_{11}^{-1}+\mathbf{b}_{vm} \cdot J_{12}^{-1}\right)\hat{\mathbf{L}}=0
\end{equation}$$


## Boundary Conditions

To get a solveable System we have to pin at least two poles at the boundary. This can be done by setting one corner-point in the L-plane to zero. If we have a developable surface we have to look which direction (u or v) is build by straight lines. With this information we can mesure the distance between two corner points and choose the x-position of the second corner pole in the L-plane to the mesured value. The y-position is set to zero. The boundary condition is applied by subtracting the choosen columnes and multiplying them by the known values and subtracting the result from the rhs.

## Solving the system

To solve the system we use a conjugate gradient method for least-square systems. This methode is provided by Eigen and does a very good job. As the matrix won't be much bigger then 100 x 100 also direct methodes can be used to solve the system. Maybe also the LDLt-solver is a good choice. 
 